In [1]:

# @markdown Name this new dataset:
dataset = '' # @param {type:'string'}

# @title ## STEP 1 : Dataset Parameters
# @markdown Select the things you want your model to detect:
# @markdown Animals:
person = False # @param {type:'boolean'}
cat = False # @param {type:'boolean'}
dog = False # @param {type:'boolean'}
horse = False # @param {type:'boolean'}
cow = False # @param {type:'boolean'}
elephant = False # @param {type:'boolean'}
bear = False # @param {type:'boolean'}
zebra = False # @param {type:'boolean'}
giraffe = False # @param {type:'boolean'}
# @markdown Vehicles:
car = False # @param {type:'boolean'}
truck = False # @param {type:'boolean'}
bus = False # @param {type:'boolean'}
boat = False # @param {type:'boolean'}
motorcycle = False # @param {type:'boolean'}
airplane = False # @param {type:'boolean'}
potted_plant = False # @param {type:'boolean'}
train = False # @param {type:'boolean'}
# @markdown Things:
toilet = False # @param {type:'boolean'}
skateboard = False # @param {type:'boolean'}
knife = False # @param {type:'boolean'}
bicycle = False # @param {type:'boolean'}
umbrella = False # @param {type:'boolean'}

# @markdown Choose the maximum number of videos you want to download:
# @markdown (some may have been taken off of YouTube, etc..)
max_videos_to_download = 0 # @param {type:'integer'}

# @markdown Enter percentage of data to use for train set:
training_percent = 70 # @param {type: 'integer'}
# @markdown What to do with the rest?
half_half = False # ['Use it all for val set', 'Half val set, half test set'] {type: 'string'}

# @markdown Shuffle videos before downloading?
# @markdown (recommended unless you want a very unbalanced dataset)
shuffle = True # @param {type:'bool'}

In [2]:
# @title ## STEP 2 : Model Selection
# @markdown Choose which size of YOLOv5 model to use:
network_size = 'small' # @param ['small', 'medium', 'large', 'xl'] {type:'string'}